In [1]:
import pandas as pd
import numpy as np
import pickle
import copy
import math
import os

In [2]:
def store_data(data, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

def get_frac_split(meta_df, matching_field, ind_column, num_folds=5):
    # Copy dataframe.
    df = meta_df.copy(deep=True)

    # Get unique classes.
    unique_classes = np.unique(meta_df[ind_column])
    # randomize rows
    df = df.sample(frac=1).reset_index(drop=True)

    folds          = dict()
    for i in range(num_folds):
        folds[i] = dict()
        folds[i]['train'] = list()
        folds[i]['test']  = list()

    for class_ in unique_classes:
        # Get slides for class.
        slides      = np.unique(df[df[ind_column]==class_][matching_field].values)

        # Test size.
        num_samples = len(slides)
        test_size   = math.floor(num_samples*(1/num_folds))

        # Iterate through chunks and add samples to fold.
        for i in range(num_folds):
            test_sample  = slides[i*test_size:(i+1)*test_size]
            train_sample = list(set(slides).difference(set(test_sample)))
            folds[i]['train'].extend(train_sample)
            folds[i]['test'].extend(test_sample)

    return folds

def get_folds(meta_df, matching_field, ind_column, num_folds=5, valid_set=False):

    # Get initial split for train/test.
    folds = get_frac_split(meta_df, matching_field, ind_column, num_folds=num_folds)

    for i in range(num_folds):
        whole_train_samples = folds[i]['train']
        subset_df = meta_df[meta_df[matching_field].isin(whole_train_samples)]
        train_val_folds = get_frac_split(subset_df, matching_field, ind_column, num_folds=num_folds)
        del folds[i]['train']
        folds[i]['train'] = train_val_folds[0]['train']
        folds[i]['valid'] = train_val_folds[0]['test']

    return folds

# Verify: This should all be empty.
def sanity_check_overlap(folds, num_folds):
    # For each fold, no overlap between cells.
    for i in range(num_folds):
        result = set(folds[i]['train']).intersection(set(folds[i]['valid']))
        if len(result) > 0:
            print(result)

        result = set(folds[i]['train']).intersection(set(folds[i]['test']))
        if len(result) > 0:
            print(result)

        result = set(folds[i]['valid']).intersection(set(folds[i]['test']))
        if len(result) > 0:
            print(result)

        # No overlap between test sets of all folds.
        for i in range(num_folds):
            for j in range(num_folds):
                if i==j: continue
                result = set(folds[i]['test']).intersection(set(folds[j]['test']))
                if len(result) > 0:
                    print('Fold %s-%s' % (i,j), result)

# Fit for legacy code.
def fit_format(folds):
    slides_folds = dict()
    for i, fold in enumerate(folds):
        slides_folds[i] = dict()
        slides_folds[i]['train'] = [(slide, None, None) for slide in folds[i]['train']]
        slides_folds[i]['valid'] = [(slide, None, None) for slide in folds[i]['valid']]
        slides_folds[i]['test']  = [(slide, None, None) for slide in folds[i]['test']]

    return slides_folds


In [7]:
main_path = '/mnt/cephfs/sharedscratch/users/fshahi/Projects/Histomorphological-Phenotype-Learning'
meta_csv   = '%s/files/Meso_patients.csv' % main_path
pickle_path = '%s/pkl_Meso.pkl' % main_path
# Read meta data file, rename column.
meta_df  = pd.read_csv(meta_csv)

In [ ]:

folds       = get_folds(meta_df, matching_field='samples', ind_column='case_Id', num_folds=5, valid_set=True)
final_folds = fit_format(folds)

# If no output, all good.
sanity_check_overlap(folds, num_folds=5)

store_data(final_folds, pickle_path)

In [10]:

# reading pkl file
def read_pkl(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data
read_pkl(pickle_path)

{0: {'train': [(1, None, None),
   (2, None, None),
   (3, None, None),
   (4, None, None),
   (5, None, None),
   (6, None, None),
   (7, None, None),
   (8, None, None),
   (9, None, None),
   (10, None, None),
   (11, None, None),
   (12, None, None),
   (13, None, None),
   (14, None, None),
   (15, None, None),
   (16, None, None),
   (17, None, None),
   (18, None, None),
   (19, None, None),
   (20, None, None),
   (21, None, None),
   (22, None, None),
   (23, None, None),
   (24, None, None),
   (25, None, None),
   (26, None, None),
   (27, None, None),
   (28, None, None),
   (29, None, None),
   (30, None, None),
   (31, None, None),
   (32, None, None),
   (33, None, None),
   (34, None, None),
   (35, None, None),
   (36, None, None),
   (37, None, None),
   (38, None, None),
   (39, None, None),
   (40, None, None),
   (41, None, None),
   (42, None, None),
   (43, None, None),
   (44, None, None),
   (45, None, None),
   (46, None, None),
   (47, None, None),
   (48, No